In [3]:
import pandas as pd

In [4]:
no_missing_merged_loc = pd.read_csv('clean_for_training.csv')
no_missing_merged_loc.drop(columns='Unnamed: 0', inplace=True)

In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.model_selection import train_test_split
import pickle
# Updated list of columns to normalize
columns_to_normalize = [
    "rooms_en_imputed", "project_count", "landmark_count", "metro_count", 
    "mall_count", "Al Makhtoum International Airport", "Burj Al Arab", 
    "Burj Khalifa", "City Centre Mirdif", "Downtown Dubai", 
    "Dubai International Airport", "Dubai Mall", "Dubai Parks and Resorts", 
    "Expo 2020 Site", "Global Village", "Hamdan Sports Complex", 
    "IMG World Adventures", "Ibn-e-Battuta Mall", "Jabel Ali", 
    "Mall of the Emirates", "Marina Mall", "Motor City", "center", 
    "east", "north", "south", "west", 
    "transaction_datetime_month", "transaction_datetime_day", 
    "transaction_datetime_weekday", "transaction_datetime_dayofyear", 
    "req_from_month", "req_from_weekday", "req_from_dayofyear", 
    "req_to_month", "req_to_day", "req_to_weekday", "req_to_dayofyear",
    "parking_count"
]

# Initialize the scaler
scaler = StandardScaler()

# Apply normalization to the specified columns
no_missing_merged_loc[columns_to_normalize] = scaler.fit_transform(no_missing_merged_loc[columns_to_normalize])
# Define features (X) and target (y)
X = no_missing_merged_loc.drop(columns=['amount'])  # Replace 'amount' with your target column if different
y = no_missing_merged_loc['amount']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Further split training data for meta-learner (optional)
X_train_base, X_val_meta, y_train_base, y_val_meta = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

# Initialize QuantileTransformers
qt_amount = QuantileTransformer(output_distribution='normal', random_state=42)
qt_size = QuantileTransformer(output_distribution='normal', random_state=42)

# Fit transformers on training data
# Fit qt_size on transaction and property sizes
qt_size.fit(X_train[['transaction_size_sqm', 'property_size_sqm']])

# Fit qt_amount on the entire `y_train` dataset
qt_amount.fit(y_train.values.reshape(-1, 1))

# Transform sizes in the training, validation, and test sets
X_train_base[['transaction_size_sqm', 'property_size_sqm']] = qt_size.transform(
    X_train_base[['transaction_size_sqm', 'property_size_sqm']]
)
X_val_meta[['transaction_size_sqm', 'property_size_sqm']] = qt_size.transform(
    X_val_meta[['transaction_size_sqm', 'property_size_sqm']]
)
X_test[['transaction_size_sqm', 'property_size_sqm']] = qt_size.transform(
    X_test[['transaction_size_sqm', 'property_size_sqm']]
)

# Transform target variable in the training, validation, and test sets
y_train_base = qt_amount.transform(y_train_base.values.reshape(-1, 1)).flatten()
y_val_meta = qt_amount.transform(y_val_meta.values.reshape(-1, 1)).flatten()
y_test = qt_amount.transform(y_test.values.reshape(-1, 1)).flatten()

# Save the transformers for later use
with open("qt_amount.pkl", "wb") as f:
    pickle.dump(qt_amount, f)
with open("qt_size.pkl", "wb") as f:
    pickle.dump(qt_size, f)

# Print dataset sizes
print(f"Training set (Base models): {X_train_base.shape}")
print(f"Validation set (Meta-learner): {X_val_meta.shape}")
print(f"Test set: {X_test.shape}")

Training set (Base models): (97683, 51)
Validation set (Meta-learner): (32561, 51)
Test set: (32562, 51)


In [7]:
from tqdm.notebook import tqdm

In [10]:
import optuna
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score, KFold
import numpy as np
import cupy
# Define XGBoost optimization with GPU support and cross-validation
def optimize_xgboost(trial, X_train, y_train):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 500),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "tree_method": "hist",  # Use "hist" for GPU
        "device": "cuda"  # Specify CUDA device for GPU training
    }
    model = XGBRegressor(**params, random_state=42)

    # 5-fold cross-validation
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(model, cupy.array(X_train), y_train, cv=kf, scoring="r2", n_jobs=-1)
    return np.mean(scores)  # Return the average R2 score

# Define Random Forest optimization with cross-validation
def optimize_random_forest(trial, X_train, y_train):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 500),
        "max_depth": trial.suggest_int("max_depth", 3, 20),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
        "n_jobs": -1  # Enable parallel processing
    }
    model = RandomForestRegressor(**params, random_state=42)

    # 5-fold cross-validation
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(model, X_train, y_train, cv=kf, scoring="r2", n_jobs=-1)
    return np.mean(scores)  # Return the average R2 score

# Define optimization function
def optimize_model(optimize_func, X_train, y_train, n_trials=20):
    def objective(trial):
        return optimize_func(trial, X_train, y_train)

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=n_trials, n_jobs=-1)  # Enable parallel optimization
    return study.best_params

# Example usage with your dataset
# Assuming X_train_base and y_train_base are defined
print("Optimizing XGBoost...")
best_params_xgb = optimize_model(optimize_xgboost, X_train_base, y_train_base)
print("Best parameters for XGBoost:", best_params_xgb)

print("Optimizing Random Forest...")
best_params_rf = optimize_model(optimize_random_forest, X_train_base, y_train_base)
print("Best parameters for Random Forest:", best_params_rf)

ModuleNotFoundError: No module named 'cupy'